In [1]:
#Done
import numpy as np
import pandas as pd
import pandas.io.sql as psql
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
import pyodbc
import requests
import json
import os


server = 'servername'
database = '[dbo]'
username = 'username'
password = 'password'
connection = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER='+
                            server + ';DATABASE=' + database +
                            ';UID=' + username + ';PWD=' + password)
cursor = connection.cursor()

query_params = {
'query': '{} -is:retweet lang:en',
'max_results': 100,
'user.fields' : 'id,location,name',
'tweet.fields' : 'public_metrics,source',
'expansions':'author_id'
}

Topic1 = 'Climate Change'
Topic1ID = 1
Topic2 = 'Electric Cars'
Topic2ID = 2
Topic3 = 'Air Pollution'
topic3ID = 3

In [22]:
def AddTopics(topic, query_params):
    query_params['query'] = query_params['query'].format(topic)
    SQL = '''
    EXEC Twitter.NewTopic ?, ?
    '''
    cursor.execute(SQL, (topic, str(query_params)))
    myID = cursor.fetchone()
    cursor.commit()
    return myID[0]

In [2]:
def GetRawTweetData(SearchPhrase, next_token):
    
    bearer_token = 'token_here'
            
    search_url = "https://api.twitter.com/2/tweets/search/recent"  
    query_params['query'] = '{} -is:retweet lang:en'.format(SearchPhrase)
    query_params['next_token'] = next_token
    headers = {
    'Authorization': 'Bearer {}'.format(bearer_token),
    'User-Agent': 'TopicPull',
    }

    response = requests.request("GET", search_url, headers=headers, params=query_params)
    json_response = response.json()
    
    
    return json_response

In [7]:
def StoreRawTweetData(Data, TopicID):
    SQL = '''
    EXEC Twitter.insTweetData ?, ?
    '''
    cursor.execute (SQL, (TopicID, Data))
    myID = cursor.fetchone()
    cursor.commit()
    return myID[0]

In [8]:
def AddUser(twitterUserID, name):
    SQL = '''
    EXEC Twitter.insUsers ? , ?
    '''
    cursor.execute(SQL, (twitterUserID, name))
    myID = cursor.fetchone()
    cursor.commit()
    return myID[0]

In [9]:
def AddSource(source):
    SQL = '''
    EXEC Twitter.addSource ?
    '''
    cursor.execute(SQL, (source))
    myID = cursor.fetchone()
    cursor.commit()
    return myID[0]

In [10]:
def AddTweet(apiTweetID, rawID, text, rtCount, replyCount, likeCount, quoteCount, location, userID, sourceID, TopicID):
    SQL = '''
    EXEC Twitter.insTweets ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?
    '''
    cursor.execute(SQL, (apiTweetID, rawID, text, rtCount, replyCount, likeCount, quoteCount, location, userID, sourceID, TopicID))
    myID = cursor.fetchone()
    cursor.commit()
    

In [29]:
def PullAndStoreTweetInfo(Topic, TopicID):
    next_token = None
    for i in range(12):

        result = GetRawTweetData(Topic, next_token)

        if('meta' in result):
            if 'next_token' in result['meta']:
                next_token = str(result['meta']['next_token'])
            else:
                next_token = None
        else:
            print('There was an error returned from the api')
            break

        for tweet, userInfo in zip(result['data'], result['includes']['users']):

            rawTweetDataID = StoreRawTweetData(json.dumps(tweet), int(Topic1ID))
            
            if(rawTweetDataID == 999999999):
                print('error inserting tweet' )
                break
            
            API_TweetID = tweet['id']

            TweetText = tweet['text']

            if('location' in userInfo):
                location = userInfo['location']
            else:
                location = 'Unknown'

            userID = AddUser(userInfo['id'], userInfo['name'])
            
            if(userID == 999999999):
                print('error inserting user')
                break

            SourceID = AddSource(tweet['source'])
            
            if(SourceID == 999999999):
                SourceID = -1

            retweetCount = tweet['public_metrics']['retweet_count']

            replyCount = tweet['public_metrics']['reply_count']

            likeCount = tweet['public_metrics']['like_count']

            quoteCount = tweet['public_metrics']['quote_count']
            
            tweetID = AddTweet(API_TweetID, rawTweetDataID, TweetText, retweetCount, replyCount, likeCount, quoteCount, location, userID, SourceID, TopicID)
            
            if(tweetID == 999999999):
                print('error inserting tweet')
                break

           
    

In [12]:
def StoreRawSentData(TweetID, rawTweetDataID, data):
    SQL = '''
    EXEC Twitter.insSentData ?, ?, ?
    '''
    cursor.execute (SQL, (TweetID, rawTweetDataID, str(data)))
    myID = cursor.fetchone()
    cursor.commit()
    return myID[0]

In [13]:
def getRawSentimentData(TweetID, tweetText):
    
    api_key = 'pvt_Key'
    api_endpoint = 'EnpointURL'
    
    api_call = TextAnalyticsClient(endpoint=api_endpoint, credential=AzureKeyCredential(api_key))
    call_id = TweetID
    call_text = tweetText
    
    api_document = []
    api_document.append({
        'id':str(call_id),
        'language':'en',
        'text':call_text
        })
    
    result = {}
    sentiment = api_call.analyze_sentiment(api_document, show_opinion_mining=True)
    result['sentiment'] = sentiment
    key_results = api_call.extract_key_phrases(api_document)
    result['key_results'] = key_results
    #returns a dictionary of 2 lists
             
            
    return result


In [34]:
def GetTweetsForAnalysis(TopicID):
    SQL = '''
    EXEC Twitter.GetTweetsForAnalysis {}
    '''
    df = pd.read_sql(SQL.format(TopicID), connection)
    df.reset_index()

    return df

In [15]:
def StoreSentiment(sentiment):
    SQL = '''
    EXEC Twitter.AddSentiment ?
    '''
    cursor.execute(SQL, (sentiment))
    myID = cursor.fetchone()
    cursor.commit()
    return myID[0]

In [113]:
def StoreTweetSentimentData(tweetID, sentimentID, pos, neg, neut):
    SQL = '''
    EXEC Twitter.insTweetSentiments ?, ?, ?, ?, ?
    '''
    cursor.execute(SQL, (tweetID, sentimentID, float(pos), float(neg), float(neut)))
    myID = cursor.fetchone()
    cursor.commit()
    return myID[0]

In [82]:
def StoreKeyPhrases(keyPhrase, TwitterID):
    SQL = '''
    EXEC Twitter.insKeyPhrases ?, ?
    '''
    cursor.execute(SQL, (keyPhrase, TwitterID))
    cursor.commit()
    #MyID = cursor.fetchone();
    #return MyID[0]


In [107]:
def AnalyzeAndStoreTweets(TopicID):
    dfTweets = GetTweetsForAnalysis(TopicID)
    #tweetTxt = idx:1,
    #tweetID = idx:2,
    #RawTweetID = idx:3,
    #TopicID = idx:4
    for item in dfTweets.itertuples():
        tweetTxt = item[1]
        tweetID = int(item[2])
        rawTweetID = int(item[3])
        TopicID = int(item[4])
        rawAnalysisData = getRawSentimentData(tweetID, tweetTxt) #1

        rawSentDataID = StoreRawSentData(tweetID, rawTweetID, str(rawAnalysisData)) #2

        SentimentData = rawAnalysisData['sentiment'][0]
        KeyPhraseData = rawAnalysisData['key_results'][0]
        sentiment = SentimentData.sentiment

        sentimentID = StoreSentiment(sentiment)#3
        if(sentimentID == 999999999):
            print('something went wrong entering the sentiment')
            break

        positive = SentimentData.confidence_scores['positive']
        negative = SentimentData.confidence_scores['negative']
        neutral = SentimentData.confidence_scores['neutral']

        TweetSentimentsID = StoreTweetSentimentData(tweetID, sentimentID, positive, negative, neutral)#4

        for phrase in KeyPhraseData.key_phrases:
            StoreKeyPhrases(phrase, tweetID)


In [ ]:
    ##Add a topic
#Topic1ID = AddTopics(Topic1, query_params)
#Topic2ID = AddTopics(Topic2, query_params)
#Topic3ID = AddTopics(Topic3, query_params)
    ##Pull, parse, and store the tweet info:
#PullAndStoreTweetInfo(Topic1, Topic1ID)
#PullAndStoreTweetInfo(Topic2, Topic2ID)
#PullAndStoreTweetInfo(Topic3, Topic3ID)
    ##Analyze, parse, and store Sentiment info:
#AnalyzeAndStoreTweets(Topic1ID)
#print("Topic1 done")
#AnalyzeAndStoreTweets(Topic2ID)
#print("Topic2 done")
#AnalyzeAndStoreTweets(Topic3ID)
#print("Topic3 done")
